# Build a simulation using the Python API

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

Currently, this notebook manipulates the simulation by directly placing objects inside of the attributes of the network and domain. It should be refactored when proper methods exist for adding these objects.

Import the Simulation class

In [ ]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [ ]:
my_sim = Simulation()
net = my_sim.network
my_sim.describe_state()

## Add nodes

In [ ]:
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server

In [ ]:
my_pc = Computer.from_config(
    config={
        "type": "computer",
        "hostname":"pc_1",
        "ip_address":"192.168.1.10",
        "subnet_mask":"255.255.255.0",
    }
   )
net.add_node(my_pc)
my_server = Server.from_config(
    config={
        "type": "server",
        "hostname":"Server",
        "ip_address":"192.168.1.11",
        "subnet_mask":"255.255.255.0"
    }
)
net.add_node(my_server)


## Connect the nodes

In [ ]:
from primaite.simulator.network.hardware.nodes.host.host_node import NIC
from primaite.simulator.network.hardware.nodes.network.switch import Switch


In [ ]:
my_switch = Switch.from_config(
    config = {
        "type":"switch",
        "hostname":"switch1",
        "num_ports":12
    }
)
net.add_node(my_switch)

pc_nic = NIC(ip_address="130.1.1.1", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_pc.connect_nic(pc_nic)

server_nic = NIC(ip_address="130.1.1.2", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_server.connect_nic(server_nic)

net.connect(pc_nic, my_switch.network_interface[1])
net.connect(server_nic, my_switch.network_interface[2])


## Add files and folders to nodes


In [ ]:
from primaite.simulator.file_system.file_type import FileType
from primaite.simulator.file_system.file_system import File
from primaite.simulator.system.core.sys_log import SysLog

In [ ]:
my_pc_downloads_folder = my_pc.file_system.create_folder("downloads")
my_pc_downloads_folder.add_file(File(name="firefox_installer.zip",folder_id="Test", folder_name="downloads" ,file_type=FileType.ZIP, sys_log=SysLog(hostname="Test")))

In [ ]:
my_server_folder = my_server.file_system.create_folder("static")
my_server.file_system.create_file("favicon.ico", file_type=FileType.PNG)

## Add applications to nodes

In [ ]:
from pydantic import Field

from pathlib import Path
from primaite.simulator.system.applications.application import Application, ApplicationOperatingState
from primaite.simulator.system.software import SoftwareHealthState, SoftwareCriticality
from primaite.simulator.file_system.file_system import FileSystem
from primaite.utils.validation.ip_protocol import PROTOCOL_LOOKUP
from primaite.utils.validation.port import PORT_LOOKUP
from primaite.simulator.system.core.sys_log import SysLog


# no applications exist yet so we will create our own.
class MSPaint(Application, discriminator="MSPaint"):
    class ConfigSchema(Application.ConfigSchema):
        type: str = "MSPaint"

    config: ConfigSchema = Field(default_factory=lambda: MSPaint.ConfigSchema())

    def __init__(self, **kwargs):
        kwargs["name"] = "MSPaint"
        kwargs["port"] = PORT_LOOKUP["HTTP"]
        kwargs["protocol"] = PROTOCOL_LOOKUP["NONE"]
        super().__init__(**kwargs)

    def describe_state(self):
        return super().describe_state()

In [ ]:
my_pc.software_manager.install(MSPaint)
mspaint = my_pc.software_manager.software.get("MSPaint")

In [ ]:
my_pc.applications[mspaint.uuid] = mspaint

## Create a domain account

In [ ]:
from primaite.simulator.domain.account import Account, AccountType


In [ ]:
acct = Account(username="admin", password="admin12", account_type=AccountType.USER)
my_sim.domain.accounts[acct.uuid] = acct

Verify that the state dictionary contains no non-serialisable objects.

In [ ]:
my_sim.describe_state()

In [ ]:
import json
json.dumps(my_sim.describe_state())